# 🏦 Large Transaction Review with Human Escalation

## Overview

This notebook demonstrates **human escalation** in financial workflows - when AI-generated responses for high-value transactions require human manager approval before execution.

### 💼 Industry Use Case: Wire Transfer Authorization

For large wire transfers above threshold limits:
1. **Transfer Agent** prepares the wire transfer details
2. **Risk Reviewer** evaluates and escalates to human manager
3. **Human Manager** reviews and approves/declines
4. Only approved transfers are processed

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational purposes only.** Actual wire transfers require secure authentication, regulatory compliance, and proper authorization controls.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **WorkflowAgent** | Wraps workflow as standard agent interface |
| **Human Escalation** | Route high-risk decisions to human managers |
| **RequestInfoExecutor** | Checkpoint for human approval |
| **Function Call Protocol** | Human feedback via typed messages |

### Architecture

```
Wire Transfer Request
    ↓
Transfer Agent (prepares details)
    ↓
Risk Reviewer (always escalates for demo)
    ↓
RequestInfoExecutor (human manager review)
    ↓
Manager approves/declines
    ↓
Transfer processed or rejected
```

## Prerequisites

- ✅ OpenAI API key configured: `OPENAI_API_KEY`
- ✅ Agent Framework installed: `pip install agent-framework`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
import sys
from collections.abc import Mapping
from dataclasses import dataclass
from pathlib import Path
from typing import Any, cast

import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential

from agent_framework import (
    ChatMessage,
    Executor,
    FunctionCallContent,
    FunctionResultContent,
    Role,
    WorkflowAgent,
    WorkflowBuilder,
    WorkflowContext,
    handler,
    response_handler,
)
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify environment is loaded
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(f"✅ Environment loaded: {azure_endpoint is not None and deployment_name is not None}")

## 2️⃣ Define Transfer and Review Message Types

Reused from the reflection pattern with FSI context.

In [ ]:
# Define message types for wire transfer review workflow

from uuid import uuid4
from agent_framework import AgentRunUpdateEvent, AgentResponseUpdate, ChatClientProtocol, Contents

@dataclass
class ReviewRequest:
    """Request for review of a prepared wire transfer."""
    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]

@dataclass
class ReviewResponse:
    """Manager's response to the wire transfer request."""
    request_id: str
    feedback: str
    approved: bool

@dataclass
class ManagerApprovalRequest:
    """Request for manager approval of a wire transfer."""
    transfer_request: ReviewRequest | None = None

class TransferAgent(Executor):
    """Executor that prepares wire transfer details and handles manager feedback."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("💸 Transfer Agent: Preparing wire transfer details...")

        # Initialize with wire transfer context
        messages = [
            ChatMessage(
                role=Role.SYSTEM, 
                text=(
                    "You are a Wire Transfer Specialist at a bank.\n"
                    "For wire transfer requests:\n"
                    "1. Validate the transfer details\n"
                    "2. Calculate any applicable fees\n"
                    "3. Provide estimated processing time\n"
                    "4. List any required documentation\n"
                    "Present the transfer summary clearly for manager approval."
                )
            )
        ]
        messages.extend(user_messages)

        print("💸 Transfer Agent: Generating transfer summary...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"💸 Transfer Agent: Summary prepared")

        messages.extend(response.messages)

        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"💸 Transfer Agent: Sending for manager review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"💸 Transfer Agent: Manager decision for {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("✅ Transfer Agent: Wire transfer APPROVED! Processing...")
            # Collect all content from agent messages
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit the approved response using AgentRunUpdateEvent
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"❌ Transfer Agent: Transfer DECLINED. Reason: {review.feedback}")
        
        # Notify about declined transfer
        decline_msg = ChatMessage(
            role=Role.ASSISTANT,
            text=f"❌ WIRE TRANSFER DECLINED\n\nReason: {review.feedback}\n\nPlease contact your manager for further assistance."
        )
        await ctx.add_event(
            AgentRunUpdateEvent(self.id, data=AgentResponseUpdate(contents=decline_msg.contents, role=Role.ASSISTANT))
        )

## 3️⃣ Define Manager Review Request

This message type wraps the transfer request for escalation to a human manager.

## 4️⃣ Create Transfer Reviewer with Human Escalation

This reviewer **always escalates** large wire transfers to a human manager. In production, you'd add logic to escalate only above threshold amounts.

### Handler Methods:

#### `review`
- Receives transfer request from Transfer Agent
- Escalates to human manager via `RequestInfoExecutor`

#### `accept_manager_decision`
- Receives manager's approval/decline decision
- Forwards decision back to Transfer Agent

In [ ]:
class TransferReviewerWithEscalation(Executor):
    """Executor that escalates wire transfers to human manager for approval."""

    def __init__(self, transfer_agent_id: str, reviewer_id: str | None = None) -> None:
        unique_id = reviewer_id or f"{transfer_agent_id}-reviewer"
        super().__init__(id=unique_id)
        self._transfer_agent_id = transfer_agent_id

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext) -> None:
        """Evaluate transfer request and escalate to human manager."""
        print(f"📋 Reviewer: Evaluating transfer request {request.request_id[:8]}...")
        print("📋 Reviewer: Wire transfer exceeds threshold - escalating to manager...")

        # Use ctx.request_info() to escalate to human manager
        await ctx.request_info(
            request_data=ManagerApprovalRequest(transfer_request=request),
            response_type=ReviewResponse
        )

    @response_handler
    async def accept_manager_decision(
        self,
        original_request: ManagerApprovalRequest,
        response: ReviewResponse,
        ctx: WorkflowContext[ReviewResponse],
    ) -> None:
        """Accept the human manager's decision and forward to Transfer Agent."""
        status = "✅ APPROVED" if response.approved else "❌ DECLINED"
        print(f"📋 Reviewer: Manager decision - {status}")
        print(f"📋 Reviewer: Manager feedback: {response.feedback}")
        print("📋 Reviewer: Forwarding decision to Transfer Agent...")
        
        await ctx.send_message(response, target_id=self._transfer_agent_id)

## 5️⃣ Build Workflow and Wrap as Agent

### Workflow Architecture:

1. **Transfer Agent ↔ Reviewer**: Bidirectional for transfer/decision flow
2. **Reviewer → RequestInfoExecutor**: Escalation to human manager
3. **RequestInfoExecutor → Reviewer**: Manager decision return
4. **`.as_agent()`**: Wraps workflow with agent interface

In [ ]:
async def run_wire_transfer_workflow() -> None:
    print("🏦 WIRE TRANSFER AUTHORIZATION WORKFLOW")
    print("=" * 60)

    # Create Azure OpenAI chat client
    print("Creating Azure OpenAI chat client...")
    chat_client = AzureOpenAIChatClient(
        credential=AzureCliCredential(),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    )
    print("✅ Azure OpenAI Chat Client created")

    print("Building workflow with Transfer Agent ↔ Reviewer ↔ Manager...")
    
    # Build workflow using register_executor pattern
    agent = (
        WorkflowBuilder()
        .register_executor(
            lambda: TransferAgent(id="transfer-agent", chat_client=chat_client),
            name="transfer_agent",
        )
        .register_executor(
            lambda: TransferReviewerWithEscalation(transfer_agent_id="transfer-agent"),
            name="reviewer",
        )
        .add_edge("transfer_agent", "reviewer")  # Transfer Agent sends to Reviewer
        .add_edge("reviewer", "transfer_agent")  # Reviewer sends decision back
        .set_start_executor("transfer_agent")
        .build()
        .as_agent()
    )
    print("✅ Workflow built with human escalation")

    # Sample wire transfer request
    wire_transfer_request = """
    WIRE TRANSFER REQUEST
    =====================
    From Account: **** 7891 (Business Checking)
    To: International Supplier Inc.
    Bank: HSBC London
    SWIFT: HSBCGB2L
    Account: GB82 HBSC 4002 0300 1234 56
    
    Amount: $75,000 USD
    Purpose: Invoice #INV-2024-0892 - Equipment Purchase
    
    Requestor: John Smith, CFO
    Contact: john.smith@company.com
    """

    print("\n📧 WIRE TRANSFER REQUEST:")
    print("-" * 60)
    print(wire_transfer_request.strip())
    print("-" * 60)
    print("\n⏳ Processing transfer request...\n")

    # Run the agent with the transfer request
    response = await agent.run(wire_transfer_request)

    # Locate the human review function call in the response
    human_review_call: FunctionCallContent | None = None
    for message in response.messages:
        for content in message.contents:
            if isinstance(content, FunctionCallContent) and content.name == WorkflowAgent.REQUEST_INFO_FUNCTION_NAME:
                human_review_call = content

    # Handle manager approval if required
    if human_review_call:
        print("\n" + "=" * 60)
        print("👨‍💼 MANAGER APPROVAL REQUIRED")
        print("=" * 60)
        
        # Parse the request arguments
        human_request_args = human_review_call.arguments
        if isinstance(human_request_args, str):
            request: WorkflowAgent.RequestInfoFunctionArgs = WorkflowAgent.RequestInfoFunctionArgs.from_json(human_request_args)
        elif isinstance(human_request_args, Mapping):
            request = WorkflowAgent.RequestInfoFunctionArgs.from_dict(dict(human_request_args))
        else:
            raise TypeError("Unexpected argument type for human review function call.")

        request_payload: Any = request.data
        
        # Handle both dataclass object and dict formats
        if isinstance(request_payload, ManagerApprovalRequest):
            # Direct dataclass object
            agent_request = request_payload.transfer_request
        elif isinstance(request_payload, Mapping):
            # Dictionary format
            transfer_request_obj = request_payload.get("transfer_request")
            if isinstance(transfer_request_obj, Mapping):
                agent_request_id = transfer_request_obj.get("request_id")
            else:
                agent_request_id = None
        else:
            # Try to access as object with attributes
            agent_request = getattr(request_payload, 'transfer_request', None)
        
        # Extract request_id
        if 'agent_request' in dir() and agent_request is not None:
            request_id = agent_request.request_id if hasattr(agent_request, 'request_id') else agent_request.get('request_id')
        elif 'agent_request_id' in dir() and agent_request_id is not None:
            request_id = agent_request_id
        else:
            # Fallback: generate a new ID
            from uuid import uuid4
            request_id = str(uuid4())
            print(f"⚠️ Could not extract request_id, using generated: {request_id[:8]}")

        # Simulate manager approval (in production, this would be actual user input)
        print("📋 Transfer details prepared by AI agent")
        print("✅ Manager decision: APPROVED")
        print("📝 Manager note: Verified against invoice and supplier records")
        
        manager_response = ReviewResponse(
            request_id=request_id, 
            feedback="Verified against invoice #INV-2024-0892. Supplier confirmed.", 
            approved=True
        )

        # Send manager decision back to workflow
        human_review_result = FunctionResultContent(
            call_id=human_review_call.call_id,
            result=manager_response,
        )
        response = await agent.run(ChatMessage(role=Role.TOOL, contents=[human_review_result]))
        
        print("\n" + "=" * 60)
        print("📤 FINAL RESULT:")
        print("=" * 60)
        print(response.messages[-1].text if response.messages else "Transfer processed")

    print("\n" + "=" * 60)
    print("✅ Wire transfer workflow complete!")
    print("\n⚠️ DISCLAIMER: This is a demonstration. Actual wire transfers")
    print("   require secure authentication and compliance controls.")

## 6️⃣ Run the Wire Transfer Workflow

In [ ]:
await run_wire_transfer_workflow()

## 📝 Key Takeaways

### Human Escalation for High-Risk FSI Operations

| Benefit | Description |
|---------|-------------|
| **Risk Management** | Human oversight for large transactions |
| **Fraud Prevention** | Manual verification of unusual requests |
| **Compliance** | Regulatory requirement for dual approval |
| **Audit Trail** | Manager decisions logged for compliance |

### Industry Use Cases for Human Escalation

- **Wire Transfers**: Above threshold amounts
- **Account Changes**: Beneficial owner updates
- **Credit Decisions**: High-risk or edge cases
- **Fraud Alerts**: Suspicious activity review
- **Complaints**: Customer escalations

### Pattern Summary

```python
# 1. Reviewer escalates to human
await ctx.send_message(
    ManagerApprovalRequest(transfer_request=request),
    target_id=self._request_info_id,
)

# 2. Agent returns function call for human input
# 3. Application collects human decision
# 4. Decision sent back via FunctionResultContent
approval_result = FunctionResultContent(
    call_id=manager_approval_call.call_id,
    result=manager_response,
)
response = await agent.run(ChatMessage(role=Role.TOOL, contents=[approval_result]))
```

### Production Considerations

- **Threshold-Based Escalation**: Only escalate above limits
- **Time-Sensitive**: Add SLAs for manager response
- **Multi-Level Approval**: Chain multiple approvers
- **Delegation**: Allow managers to delegate authority